# Named Entity Extraction Tutorial
This tutorial is a slight modification of the tutorial by Sam Galen.

In [149]:
from __future__ import print_function
from sklearn.metrics import confusion_matrix
import io
import nltk
import scipy
import codecs
import sklearn
import pycrfsuite
import pandas as pd
from itertools import chain
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

print('sklearn version:', sklearn.__version__)
print('Libraries succesfully loaded!')


sklearn version: 0.20.1
Libraries succesfully loaded!


In [150]:
def sent2features(sent, feature_func):
    return [feature_func(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [s[-1] for s in sent]

def sent2tokens(sent):
    return [s[0] for s in sent]

def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(y_true)
    y_pred_combined = lb.transform(y_pred)
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )
            
def word2simple_features(sent, i):
    '''
    This makes a simple baseline.  
    You can add and/or remove features to get (much?) better results.
    Experiment with it as you will need to do this for assignment.
    '''
    word = sent[i][0]
    
    features = {
        'bias': 1.0, # This feature is constant for all words.
        'word.lower()': word.lower(), # This feature is the word, ignoring case.
        'word[-2:]': word[-2:], # This feature is the last two characters of the word (i.e. the suffix).
    }
    if i == 0:
        features['BOS'] = True # Mark the beginning of sentence.
        
    if i == len(sent)-1:
        features['EOS'] = True # Mark the end of sentence.

    return features

# load data and preprocess
def extract_data(path):
    """
    Extracting data from train file or test file. 
    path - the path of the file to extract
    
    return:
        res - a list of sentences, each sentence is a
              a list of tuples. For train file, each tuple
              contains token and label. For test file, each
              tuple only contains token.
        ids - a list of ids for the corresponding token. This
              is mainly for Kaggle submission.
    """
    file = io.open(path, mode="r", encoding="utf-8")
    next(file)
    res = []
    ids = []
    sent = []
    for line in file:
        if line != '\n':
            # Each line contains the position ID, the token, and (for the training set) the label.
            parts = line.strip().split(' ')
            sent.append(tuple(parts[1:]))
            ids.append(parts[0])
        else:
            res.append(sent)
            sent = []
                
    return res, ids
            

# Build a NER classifier

## Load data and extract features

In [151]:
# Load train and test data
train_data, train_ids = extract_data('train')
test_data, test_ids = extract_data('test')

# Load true labels for test data
test_labels = list(pd.read_csv('test_ground_truth').loc[:, 'label'])

print('Train and Test data loaded succesfully!')

# Feature extraction using the word2simple_features function
train_features = [sent2features(s, feature_func=word2simple_features) for s in train_data]
train_labels = [sent2labels(s) for s in train_data]
test_features = [sent2features(s, feature_func=word2simple_features) for s in test_data]

trainer = pycrfsuite.Trainer(verbose=False)
for xseq, yseq in zip(train_features, train_labels):
    trainer.append(xseq, yseq)
print('Feature Extraction done!')    

# Explore the extracted features    
sent2features(train_data[0], word2simple_features)

Train and Test data loaded succesfully!
Feature Extraction done!


[{'bias': 1.0, 'word.lower()': 'también', 'word[-2:]': 'én', 'BOS': True},
 {'bias': 1.0, 'word.lower()': 'el', 'word[-2:]': 'el'},
 {'bias': 1.0, 'word.lower()': 'secretario', 'word[-2:]': 'io'},
 {'bias': 1.0, 'word.lower()': 'general', 'word[-2:]': 'al'},
 {'bias': 1.0, 'word.lower()': 'de', 'word[-2:]': 'de'},
 {'bias': 1.0, 'word.lower()': 'la', 'word[-2:]': 'la'},
 {'bias': 1.0, 'word.lower()': 'asociación', 'word[-2:]': 'ón'},
 {'bias': 1.0, 'word.lower()': 'española', 'word[-2:]': 'la'},
 {'bias': 1.0, 'word.lower()': 'de', 'word[-2:]': 'de'},
 {'bias': 1.0, 'word.lower()': 'operadores', 'word[-2:]': 'es'},
 {'bias': 1.0, 'word.lower()': 'de', 'word[-2:]': 'de'},
 {'bias': 1.0, 'word.lower()': 'productos', 'word[-2:]': 'os'},
 {'bias': 1.0, 'word.lower()': 'petrolíferos', 'word[-2:]': 'os'},
 {'bias': 1.0, 'word.lower()': ',', 'word[-2:]': ','},
 {'bias': 1.0, 'word.lower()': 'aurelio', 'word[-2:]': 'io'},
 {'bias': 1.0, 'word.lower()': 'ayala', 'word[-2:]': 'la'},
 {'bias': 1.

## Explore the classifier parameters

In [159]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

## Set the classifier parameters

In [160]:
trainer.set_params({
    'c1': 0.3,   # coefficient for L1 penalty
    'c2': 1e-2,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier
    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

## Train a NER model

In [161]:
%%time
trainer.train('ner-esp.model')

print('Training done :)')

Training done :)
Wall time: 13.7 s


## Make predictions with your NER model
Make predictions and evaluate your model on the test set.
To use your NER model, create pycrfsuite.Tagger, open the model, and use the "tag" method, as follows:

In [162]:
# Make predictions
tagger = pycrfsuite.Tagger()
tagger.open('ner-esp.model')
test_pred = [tagger.tag(xseq) for xseq in test_features]
test_pred = [s for w in test_pred for s in w]

## Print evaluation
print(bio_classification_report(test_pred, test_labels))


              precision    recall  f1-score   support

       B-LOC       0.74      0.80      0.77      1879
       I-LOC       0.54      0.73      0.62       563
      B-MISC       0.36      0.62      0.46       511
      I-MISC       0.41      0.41      0.41      1235
       B-ORG       0.71      0.84      0.77      2720
       I-ORG       0.62      0.67      0.64      2057
       B-PER       0.78      0.89      0.83      1651
       I-PER       0.85      0.89      0.87      1559

   micro avg       0.67      0.76      0.71     12175
   macro avg       0.63      0.73      0.67     12175
weighted avg       0.67      0.76      0.71     12175
 samples avg       0.08      0.08      0.08     12175



In [163]:
print (len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

100 {'num': 100, 'scores': {}, 'loss': 23557.287174, 'feature_norm': 347.758296, 'error_norm': 396.576613, 'active_features': 14089, 'linesearch_trials': 1, 'linesearch_step': 1.0, 'time': 0.113}


## Check what the classifier has learned

In [164]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
I-ORG  -> I-ORG   8.446236
B-MISC -> I-MISC  8.444787
B-PER  -> I-PER   7.850296
O      -> B-ORG   7.657108
B-LOC  -> I-LOC   7.654752
B-ORG  -> I-ORG   7.461336
I-MISC -> I-MISC  7.182685
I-LOC  -> I-LOC   6.983064
I-PER  -> I-PER   6.862758
O      -> B-MISC  5.301844
O      -> B-PER   5.160665
O      -> B-LOC   4.305234
O      -> O       3.864672
B-LOC  -> B-LOC   1.842234
B-LOC  -> B-ORG   1.704278

Top unlikely transitions:
B-PER  -> O       -0.660255
B-ORG  -> B-ORG   -0.683587
B-ORG  -> B-LOC   -0.713435
B-PER  -> B-LOC   -0.760192
I-ORG  -> O       -0.840991
I-LOC  -> O       -0.932838
B-ORG  -> O       -1.009612
I-ORG  -> B-MISC  -1.091496
I-MISC -> B-PER   -1.270028
B-ORG  -> B-MISC  -1.428103
I-MISC -> O       -2.088220
I-LOC  -> B-LOC   -2.273365
B-PER  -> B-PER   -3.419571
I-MISC -> B-LOC   -3.462672
I-ORG  -> B-LOC   -3.647404


We can see that, for example, it is very likely that the beginning of a person name (B-PER) will be followed by a token inside person name (I-PER). Also note O -> B-LOC are penalized.

## Check the state features

In [165]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
10.596990 B-MISC word.lower():fomentó
10.595833 B-MISC word.lower():ss&p500
10.569087 B-LOC  word.lower():celrá
10.491885 B-ORG  word.lower():autostrade
10.028637 B-ORG  word.lower():lycos
9.919296 I-PER  word.lower():gándara
9.868742 I-MISC word.lower():extraviado
9.685962 B-LOC  word.lower():caracas
9.683591 I-LOC  word.lower():llanada
9.603334 I-PER  word.lower():sota
9.558908 B-LOC  word.lower():bruselas
9.397699 B-LOC  word.lower():portugal
9.334681 B-PER  word.lower():rivaldo
9.328376 B-LOC  word.lower():burgos
9.289254 I-ORG  word.lower():tecos
9.265410 B-ORG  word.lower():psoe-progresistas
9.255296 B-PER  word.lower():reyes
9.245421 I-PER  word.lower():zotoluco
9.179202 B-PER  word.lower():ballesteros
9.127152 B-LOC  word.lower():ceuta

Top negative:
-3.630440 O      word[-2:]:ak
-3.670430 O      word.lower():república
-3.694552 O      word[-2:]:at
-3.744290 O      word[-2:]:rt
-3.813656 O      word.lower():iglesias
-3.820395 O      word.lower():cortes
-3.826779 O